In [7]:
import torch
import torch.nn as nn
import numpy as np

# Custom Dropout class (same as in the previous example)
class Dropout(nn.Module):
    def __init__(self, p=0.5):
        super(Dropout, self).__init__()
        self.p = p
        self.training = True

    def forward(self, x):
        if self.training:
            mask = torch.bernoulli(torch.ones_like(x) * (1 - self.p))
            return x * mask / (1 - self.p)
        else:
            return x

# Synthetic dataset generation
def generate_synthetic_data(num_samples, num_features, num_classes):
    X = np.random.randn(num_samples, num_features).astype(np.float32)
    y = np.random.randint(0, num_classes, num_samples)
    return torch.from_numpy(X), torch.tensor(y)

# Model definition
class SimpleModel(nn.Module):
    def __init__(self, input_dim, hidden_dim, output_dim, dropout_rate):
        super(SimpleModel, self).__init__()
        self.fc1 = nn.Linear(input_dim, hidden_dim)
        self.dropout = Dropout(dropout_rate)
        self.fc2 = nn.Linear(hidden_dim, output_dim)

    def forward(self, x):
        x = self.fc1(x)
        x = self.dropout(x)
        x = torch.relu(x)
        x = self.fc2(x)
        return torch.log_softmax(x, dim=1)

# Training loop
def train_model(model, criterion, optimizer, num_epochs, X_train, y_train):
    model.train()
    for epoch in range(num_epochs):
        optimizer.zero_grad()
        outputs = model(X_train)
        loss = criterion(outputs, y_train)
        loss.backward()
        optimizer.step()
    return model

# Accuracy calculation
def calculate_accuracy(model, X_test, y_test):
    model.eval()
    with torch.no_grad():
        outputs = model(X_test)
        _, predicted = torch.max(outputs, 1)
        accuracy = (predicted == y_test).float().mean()
    return accuracy.item()

# Main execution
if __name__ == "__main__":
    # Hyperparameters
    input_dim = 10
    hidden_dim = 20
    output_dim = 3
    dropout_rate = 0.3
    num_epochs = 100
    batch_size = 32
    
    # Generate synthetic data
    X_train, y_train = generate_synthetic_data(500, input_dim, output_dim)
    X_test, y_test = generate_synthetic_data(100, input_dim, output_dim)
    
    # Model creation
    model_without_dropout = SimpleModel(input_dim, hidden_dim, output_dim, 0)
    model_with_dropout = SimpleModel(input_dim, hidden_dim, output_dim, dropout_rate)
    
    # Training
    criterion = nn.NLLLoss()
    optimizer = torch.optim.Adam(model_without_dropout.parameters(), lr=0.01)
    
    print("Training model without dropout...")
    model_without_dropout = train_model(model_without_dropout, criterion, optimizer, num_epochs, X_train, y_train)
    
    print("\nTraining model with dropout...")
    model_with_dropout = train_model(model_with_dropout, criterion, optimizer, num_epochs, X_train, y_train)
    
    # Accuracy calculation
    accuracy_without_dropout = calculate_accuracy(model_without_dropout, X_test, y_test)
    accuracy_with_dropout = calculate_accuracy(model_with_dropout, X_test, y_test)
    
    print(f"\nAccuracy without dropout: {accuracy_without_dropout:.4f}")
    print(f"Accuracy with dropout: {accuracy_with_dropout:.4f}")

    # Compare accuracies
    if accuracy_with_dropout > accuracy_without_dropout:
        print("\nDropout improved accuracy!")
    else:
        print("\nDropout did not improve accuracy.")


Training model without dropout...

Training model with dropout...

Accuracy without dropout: 0.2800
Accuracy with dropout: 0.3800

Dropout improved accuracy!
